In [40]:
import pandas as pd
from pathlib import Path
from os import listdir
import os.path
from collections import defaultdict
import re

In [7]:
repo_dir = Path("/home/luk/repos/RAPid-Learn/data")
experiment_id = "2267a0875d2e49f5b1f910d9b0f7df0b-2021-09-30_22:10:50-PPO-10000episodes-rewardshapingon"
experiment_path = repo_dir / experiment_id

In [15]:
novelties = ["axefirecteasy", "prenovelty", "axetobreakeasy", "axetobreakhard", "firecraftingtableeasy",
             "firecraftingtablehard", "rubbertree", "rubbertreehard", "scrapeplank"]

In [34]:
csvs_per_novelty = defaultdict(list)
for novelty in novelties:
    for trial_dir in listdir(experiment_path / novelty):
        if os.path.isdir(experiment_path / novelty / trial_dir):
            for filename in os.listdir(experiment_path / novelty / trial_dir):
                if filename.endswith(".csv"):
                    csvs_per_novelty[novelty].append(experiment_path / novelty / trial_dir / filename)
                                       

In [50]:
dfs = []
for novelty in novelties:
    for csv_path in csvs_per_novelty[novelty]:
        df = pd.read_csv(csv_path, skiprows=[0])
        df["trial"] = extract_number(str(os.path.basename(os.path.normpath(csv_path))))
        df["novelty"] = novelty
        dfs.append(df)
results_df = pd.concat(dfs).reset_index()

In [51]:
results_df

,index,r,l,t,success,mode,episode_counter,trial,novelty
0,0,-500,500,0.586607,False,learn,1.0,4,axefirecteasy
1,1,-500,500,1.158254,False,learn,2.0,4,axefirecteasy
2,2,-500,500,1.696336,False,learn,3.0,4,axefirecteasy
3,3,-500,500,2.235418,False,learn,4.0,4,axefirecteasy
4,4,-500,500,4.197490,False,learn,5.0,4,axefirecteasy
...,...,...,...,...,...,...,...,...,...
494095,11085,1478,22,1439.693673,True,eval,10001.0,5,scrapeplank
494096,11086,0,500,1440.323255,False,eval,10001.0,5,scrapeplank
494097,11087,1478,22,1440.360133,True,eval,10001.0,5,scrapeplank
494098,11088,1481,19,1440.393241,True,eval,10001.0,5,scrapeplank


In [38]:
def extract_number(f):
    s = re.findall("\d+", f)
    return int(s[0]) if s else -1

In [63]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(results_df.loc[results_df['mode'] == "eval"].groupby(by=["novelty", "episode_counter"])
      .agg({'success': ['mean', 'std'], 'r': ['mean', 'std']}).reset_index())

                   novelty episode_counter success                   r  \
                                              mean       std      mean   
0            axefirecteasy           102.0   0.000  0.000000  -293.000   
1            axefirecteasy           203.0   0.000  0.000000  -163.000   
2            axefirecteasy           304.0   0.000  0.000000  -106.000   
3            axefirecteasy           405.0   0.000  0.000000  -105.000   
4            axefirecteasy           506.0   0.000  0.000000  -107.000   
5            axefirecteasy           607.0   0.000  0.000000  -103.000   
6            axefirecteasy           708.0   0.000  0.000000  -102.000   
7            axefirecteasy           809.0   0.000  0.000000  -106.000   
8            axefirecteasy           910.0   0.000  0.000000  -102.000   
9            axefirecteasy          1011.0   0.000  0.000000  -107.000   
10           axefirecteasy          1112.0   0.000  0.000000  -100.000   
11           axefirecteasy          12

In [ ]:
TARGET_DIR = repo_dir / experiment_id / "parsed_results"
TARGET_DIR.mkdir(parents=True, exist_ok=True)

In [14]:
for csv in csvs:
    print(f"Now parsing results for {csv}!")
    df = pd.read_csv(repo_dir / experiment_id / csv, skiprows=[0])
    output = parse_results(df)
    output.to_csv(TARGET_DIR / f"parsed-{csv}")

Now parsing results for axetobreakhard-monitor.csv!
Now parsing results for axefirecteasy-monitor.csv!
Now parsing results for scrapeplank-monitor.csv!
Now parsing results for rubbertreehard-monitor.csv!
Now parsing results for firecraftingtablehard-monitor.csv!
Now parsing results for rubbertree-monitor.csv!
Now parsing results for firecraftingtableeasy-monitor.csv!
Now parsing results for axetobreakeasy-monitor.csv!
Now parsing results for prenovelty-monitor.csv!
